<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/B_adapter_seq_bn_P_2_C_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This takes an adapter type and creates a pretraining adapter, a classification adapter with no pretrainng and a classification adapter from the pretraining adapter





# Setup

In [1]:
# Pretraining Dataset Settings
pretrain_dataset_name = "BigTMiami/amazon_split_25M_reviews_20_percent_condensed"
# 5% of 20% creates a 1% - I think this should take 15 minutes
pretrain_percentage = "10"
pretrain_train_split = f"train[:{pretrain_percentage}%]"
pretrain_eval_split = "validation"

# Pretraining Adapter settings
experiment_tag = "B"
adapter_short_desc = "full"
adapter_type = "seq_bn" # could be "lora", etc.

pretrained_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_pretraining_P_{pretrain_percentage}"
pretrained_adapter_name = f"{pretrained_adapter_hub_name}_adapter"
print(f"Pretrained Adapter Hub Name: {pretrained_adapter_hub_name}")

# Classification Dataset Settings
classification_dataset_name = "BigTMiami/amazon_helpfulness"
classification_train_percentage = "10"
classification_train_split = f"train[:{classification_train_percentage}%]"
classification_eval_split = "dev"

classification_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_classification_C_{classification_train_percentage}"
classification_adapter_name = f"{classification_adapter_hub_name}_adapter"
print(f"Classifer Not Pretrained Adapter Hub Name: {classification_adapter_hub_name}")

# Classification using Pretrained Settings
classification_adapter_from_pretrained_hub_name = f"{experiment_tag}_adapter_{adapter_type}_classification_P_{pretrain_percentage}_to_C_{classification_train_percentage}"
classification_adapter_from_pretrained_name = f"{classification_adapter_from_pretrained_hub_name}_adapter"
print(f"Classifer from Pretrained Adapter Hub Name: {classification_adapter_from_pretrained_hub_name}")


Pretrained Adapter Hub Name: B_adapter_seq_bn_pretraining_P_10
Classifer Not Pretrained Adapter Hub Name: B_adapter_seq_bn_classification_C_10
Classifer from Pretrained Adapter Hub Name: B_adapter_seq_bn_classification_P_10_to_C_10


In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [7]:
from datasets import load_dataset

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=[pretrain_train_split,pretrain_eval_split])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/862683 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8360 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 86268
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8360
})


In [11]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=adapter_type)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")

Pretrained Adapter Name: B_adapter_seq_bn_pretraining_P_10_adapter


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 448 MB


In [14]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [15]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir=pretrained_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=11,
    weight_decay=0.01,
    logging_steps=200,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [16]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [17]:
before_pretrain_eval_results = pretrain_trainer.evaluate()
before_pretrain_eval_results_loss = before_pretrain_eval_results["eval_loss"]
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(before_pretrain_eval_results )

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Before pretraining loss: 16.5676
{'eval_loss': 16.567617416381836, 'eval_runtime': 91.677, 'eval_samples_per_second': 91.19, 'eval_steps_per_second': 2.858}


In [19]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

Step,Training Loss
200,3.838400


TrainOutput(global_step=245, training_loss=3.6418449557557397, metrics={'train_runtime': 1900.711, 'train_samples_per_second': 45.387, 'train_steps_per_second': 0.129, 'total_flos': 2.607358634754048e+16, 'train_loss': 3.6418449557557397, 'epoch': 1.0})


In [20]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 2.5952
{'eval_loss': 2.5951969623565674, 'eval_runtime': 90.957, 'eval_samples_per_second': 91.912, 'eval_steps_per_second': 2.88, 'epoch': 1.0}


In [21]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/B_adapter_seq_bn_pretraining_P_10/commit/1c195834066b51beadb5f5591ac5cf69a6b6b9ef', commit_message='Upload model', commit_description='', oid='1c195834066b51beadb5f5591ac5cf69a6b6b9ef', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
import gc

print_gpu_utilization()

del pretraining_data_collator
del pretrain_train_dataset
del pretrain_eval_dataset
del pretrain_model
del pretrain_trainer
gc.collect()

print_gpu_utilization()

GPU 27307 MB
GPU 27307 MB


# Classification

In [23]:
from datasets import load_dataset

classification_train_dataset, classification_eval_dataset= load_dataset(classification_dataset_name, split=[classification_train_split,classification_eval_split])

print(classification_train_dataset)
print(classification_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 11525
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [24]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_no_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# Add a new adapter
classficiation_no_pretrain_model.add_adapter(classification_adapter_name, config=adapter_type)

# Add head for classification modeling
classficiation_no_pretrain_model.add_classification_head(
    classification_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_no_pretrain_model.train_adapter(classification_adapter_name)

print(f"Classification Adapter Name: {classification_adapter_name}")

Classification Adapter Name: B_adapter_seq_bn_classification_C_10_adapter


In [26]:
from transformers import DataCollatorWithPadding

classification_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
from transformers import TrainingArguments

classification_training_args = TrainingArguments(
    output_dir=classification_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [28]:
from adapters import AdapterTrainer

classification_trainer = AdapterTrainer(
    model=classficiation_no_pretrain_model,
    args=classification_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [29]:
classification_training_results = classification_trainer.train()
print(classification_training_results)

Step,Training Loss
250,0.403600
500,0.360700
750,0.345200
1000,0.334900
1250,0.346000
1500,0.330000
1750,0.316900
2000,0.335000


TrainOutput(global_step=2163, training_loss=0.34421218328877173, metrics={'train_runtime': 553.2756, 'train_samples_per_second': 62.491, 'train_steps_per_second': 3.909, 'total_flos': 8682044212018110.0, 'train_loss': 0.34421218328877173, 'epoch': 3.0})


In [30]:
classification_eval_results = classification_trainer.evaluate()
classification_eval_loss = classification_eval_results["eval_loss"]
classification_eval_f1 = classification_eval_results["eval_f1_macro"]
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.2f}")
print(classification_eval_results)

Classification no Pretraining Eval F1:60.79
Classification no Pretraining Eval Loss: 0.33
{'eval_loss': 0.3320135176181793, 'eval_accuracy': 0.863, 'eval_f1_macro': 0.6078952607530862, 'eval_runtime': 37.6478, 'eval_samples_per_second': 132.81, 'eval_steps_per_second': 8.314, 'epoch': 3.0}


In [31]:
classficiation_no_pretrain_model.push_adapter_to_hub(
    classification_adapter_hub_name,
    classification_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_hub_name}")

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

Pushed B_adapter_seq_bn_classification_C_10


In [32]:
import gc

print_gpu_utilization()

del classficiation_no_pretrain_model
del classification_trainer
gc.collect()

print_gpu_utilization()

GPU 10493 MB
GPU 10493 MB


# Classification from Pre-training

In [48]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_from_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
load_name = f"BigTMiami/{pretrained_adapter_hub_name}"

# Load Pretrained adapter without head
loaded_adapter_name = classficiation_from_pretrain_model.load_adapter(load_name, with_head=False)

# Add head for classification modeling
classficiation_from_pretrain_model.add_classification_head(
    loaded_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_from_pretrain_model.train_adapter(loaded_adapter_name)

print(f"Classification Adapter Name: {loaded_adapter_name}")

classficiation_from_pretrain_model.active_adapters

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Classification Adapter Name: B_adapter_seq_bn_pretraining_P_10_adapter


Stack[B_adapter_seq_bn_pretraining_P_10_adapter]

In [50]:
from transformers import TrainingArguments

classification_from_pretrained_training_args = TrainingArguments(
    output_dir=classification_adapter_from_pretrained_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [51]:
from adapters import AdapterTrainer

classification_from_pretrained_trainer = AdapterTrainer(
    model=classficiation_from_pretrain_model,
    args=classification_from_pretrained_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

GPU 10493 MB


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [52]:
classification_from_pretrained_training_results = classification_from_pretrained_trainer.train()
print(classification_from_pretrained_training_results)
print_gpu_utilization()

Step,Training Loss
250,0.386500
500,0.359800
750,0.340100
1000,0.331100
1250,0.343000
1500,0.329000
1750,0.314000
2000,0.330000


TrainOutput(global_step=2163, training_loss=0.33958581438562796, metrics={'train_runtime': 553.23, 'train_samples_per_second': 62.497, 'train_steps_per_second': 3.91, 'total_flos': 8682044212018110.0, 'train_loss': 0.33958581438562796, 'epoch': 3.0})
GPU 9977 MB


In [53]:
classification_from_pretrained_eval_results = classification_from_pretrained_trainer.evaluate()
classification_from_pretrained_eval_loss = classification_from_pretrained_eval_results["eval_loss"]
classification_from_pretrained_eval_f1 = classification_from_pretrained_eval_results["eval_f1_macro"]
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.2f}")
print(classification_from_pretrained_eval_results)
print_gpu_utilization()

Classification with Pretraining Eval F1:62.51
Classification with Pretraining Eval Loss: 0.33
{'eval_loss': 0.33185887336730957, 'eval_accuracy': 0.8642, 'eval_f1_macro': 0.6251083955732535, 'eval_runtime': 37.6476, 'eval_samples_per_second': 132.811, 'eval_steps_per_second': 8.314, 'epoch': 3.0}
GPU 9977 MB


In [54]:
classficiation_from_pretrain_model.push_adapter_to_hub(
    classification_adapter_from_pretrained_hub_name,
    loaded_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_from_pretrained_hub_name}")

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

Pushed B_adapter_seq_bn_classification_P_10_to_C_10


In [55]:
print_gpu_utilization()

GPU 9977 MB


In [56]:
print("Summary")
print(f"Adapter Type:{adapter_type}")
print(f"Pretrain Train Percentage:{pretrain_percentage}")
print(f"Classification Train Percentage:{classification_train_percentage}")

print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")

print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.2f}")

print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.2f}")

pretraining_f1_improvement = (100.0 * ( classification_eval_f1- classification_from_pretrained_eval_f1))
print(f"Pretraining F1 Improvement: {pretraining_f1_improvement:.2f}")

Summary
Adapter Type:seq_bn
Pretrain Train Percentage:10
Classification Train Percentage:10
Before pretraining loss: 16.5676
After pretraining loss: 2.5952
Classification no Pretraining Eval F1:60.79
Classification no Pretraining Eval Loss: 0.33
Classification with Pretraining Eval F1:62.51
Classification with Pretraining Eval Loss: 0.33
Pretraining F1 Improvement: -1.72


In [57]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
